In [11]:
import gym
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

In [12]:

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def epsilon_action(epsilon):
    random_value = np.random.random()
    if random_value<=epsilon:
        action = env.action_space.sample()
    else:
        Y=online_model.predict(state.reshape(1,-1))
        action = np.argmax(Y[0])
    return action


In [13]:
1-0.001*500

0.5

In [16]:




gamma=0.99
epsilon_decay = 0.005
learning_rate =0.0001
max_e=1
min_e=0.0001
replay_batch_size=50
iterations=1000 

In [17]:
env = gym.make('LunarLander-v2')
state=env.reset()
online_model = initiate_NN(learning_rate)
memory=[]
episode_reward_rolling = []
episode_reward=[]

for episode in range(iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                y[non_terminal_samples] += np.multiply(gamma, \
                            online_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)

    episode_reward.append(current_episode_reward) 
    if len(memory)>=5000:
        memory = memory[-5000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  0  episode reward:  -62.22  rolling reward:  -62.22  Steps:  171  epsilon:  1.00
episode:  1  episode reward:  -114.02  rolling reward:  -88.12  Steps:  96  epsilon:  1.00
episode:  2  episode reward:  -85.90  rolling reward:  -87.38  Steps:  74  epsilon:  0.99
episode:  3  episode reward:  -163.26  rolling reward:  -106.35  Steps:  87  epsilon:  0.99
episode:  4  episode reward:  -153.40  rolling reward:  -115.76  Steps:  77  epsilon:  0.98
episode:  5  episode reward:  -179.05  rolling reward:  -126.31  Steps:  91  epsilon:  0.98
episode:  6  episode reward:  -134.49  rolling reward:  -127.48  Steps:  78  epsilon:  0.97
episode:  7  episode reward:  -236.93  rolling reward:  -141.16  Steps:  100  epsilon:  0.97
episode:  8  episode reward:  -366.27  rolling reward:  -166.17  Steps:  84  epsilon:  0.96
episode:  9  episode reward:  -344.38  rolling reward:  -183.99  Steps:  107  epsilon:  0.96
episode:  10  episode reward:  -513.35  rolling reward:  -213.93  Steps:  99  epsi

episode:  89  episode reward:  -60.47  rolling reward:  -162.45  Steps:  126  epsilon:  0.64
episode:  90  episode reward:  -60.92  rolling reward:  -161.33  Steps:  154  epsilon:  0.64
episode:  91  episode reward:  -112.15  rolling reward:  -160.80  Steps:  139  epsilon:  0.63
episode:  92  episode reward:  -94.19  rolling reward:  -160.08  Steps:  82  epsilon:  0.63
episode:  93  episode reward:  -69.85  rolling reward:  -159.12  Steps:  246  epsilon:  0.63
episode:  94  episode reward:  -27.50  rolling reward:  -157.73  Steps:  106  epsilon:  0.63
episode:  95  episode reward:  -83.91  rolling reward:  -156.97  Steps:  137  epsilon:  0.62
episode:  96  episode reward:  -60.60  rolling reward:  -155.97  Steps:  153  epsilon:  0.62
episode:  97  episode reward:  -167.25  rolling reward:  -156.09  Steps:  160  epsilon:  0.62
episode:  98  episode reward:  17.23  rolling reward:  -154.34  Steps:  84  epsilon:  0.61
episode:  99  episode reward:  -221.85  rolling reward:  -155.01  Steps

episode:  177  episode reward:  -55.12  rolling reward:  -92.23  Steps:  1000  epsilon:  0.41
episode:  178  episode reward:  -100.41  rolling reward:  -89.55  Steps:  137  epsilon:  0.41
episode:  179  episode reward:  -252.42  rolling reward:  -91.65  Steps:  870  epsilon:  0.41
episode:  180  episode reward:  51.56  rolling reward:  -89.36  Steps:  1000  epsilon:  0.41
episode:  181  episode reward:  -64.82  rolling reward:  -89.53  Steps:  212  epsilon:  0.40
episode:  182  episode reward:  -108.09  rolling reward:  -90.10  Steps:  1000  epsilon:  0.40
episode:  183  episode reward:  -89.28  rolling reward:  -90.24  Steps:  379  epsilon:  0.40
episode:  184  episode reward:  -27.92  rolling reward:  -89.31  Steps:  260  epsilon:  0.40
episode:  185  episode reward:  -42.90  rolling reward:  -89.03  Steps:  169  epsilon:  0.40
episode:  186  episode reward:  -95.06  rolling reward:  -89.58  Steps:  114  epsilon:  0.39
episode:  187  episode reward:  -105.90  rolling reward:  -89.39 

episode:  265  episode reward:  -73.49  rolling reward:  -53.90  Steps:  229  epsilon:  0.27
episode:  266  episode reward:  -48.41  rolling reward:  -54.37  Steps:  403  epsilon:  0.26
episode:  267  episode reward:  93.23  rolling reward:  -52.18  Steps:  1000  epsilon:  0.26
episode:  268  episode reward:  194.59  rolling reward:  -50.18  Steps:  788  epsilon:  0.26
episode:  269  episode reward:  -97.12  rolling reward:  -50.61  Steps:  158  epsilon:  0.26
episode:  270  episode reward:  -16.12  rolling reward:  -49.97  Steps:  448  epsilon:  0.26
episode:  271  episode reward:  89.12  rolling reward:  -46.47  Steps:  1000  epsilon:  0.26
episode:  272  episode reward:  96.21  rolling reward:  -45.40  Steps:  1000  epsilon:  0.26
episode:  273  episode reward:  -64.99  rolling reward:  -43.97  Steps:  396  epsilon:  0.26
episode:  274  episode reward:  -64.57  rolling reward:  -43.24  Steps:  126  epsilon:  0.25
episode:  275  episode reward:  17.34  rolling reward:  -41.59  Steps:

episode:  354  episode reward:  -78.08  rolling reward:  48.25  Steps:  392  epsilon:  0.17
episode:  355  episode reward:  -125.37  rolling reward:  47.62  Steps:  137  epsilon:  0.17
episode:  356  episode reward:  -6.11  rolling reward:  47.84  Steps:  234  epsilon:  0.17
episode:  357  episode reward:  -144.19  rolling reward:  47.20  Steps:  65  epsilon:  0.17
episode:  358  episode reward:  240.41  rolling reward:  48.70  Steps:  397  epsilon:  0.17
episode:  359  episode reward:  -74.87  rolling reward:  48.63  Steps:  201  epsilon:  0.17
episode:  360  episode reward:  95.90  rolling reward:  49.83  Steps:  1000  epsilon:  0.17
episode:  361  episode reward:  124.04  rolling reward:  51.17  Steps:  967  epsilon:  0.16
episode:  362  episode reward:  197.38  rolling reward:  52.68  Steps:  744  epsilon:  0.16
episode:  363  episode reward:  -107.73  rolling reward:  52.11  Steps:  1000  epsilon:  0.16
episode:  364  episode reward:  -16.13  rolling reward:  51.73  Steps:  1000  

episode:  443  episode reward:  217.27  rolling reward:  116.74  Steps:  682  epsilon:  0.11
episode:  444  episode reward:  105.66  rolling reward:  115.79  Steps:  1000  epsilon:  0.11
episode:  445  episode reward:  265.06  rolling reward:  116.25  Steps:  298  epsilon:  0.11
episode:  446  episode reward:  299.08  rolling reward:  118.80  Steps:  480  epsilon:  0.11
episode:  447  episode reward:  235.65  rolling reward:  121.23  Steps:  836  epsilon:  0.11
episode:  448  episode reward:  241.09  rolling reward:  121.64  Steps:  263  epsilon:  0.11
episode:  449  episode reward:  261.30  rolling reward:  123.94  Steps:  324  epsilon:  0.11
episode:  450  episode reward:  249.54  rolling reward:  126.16  Steps:  342  epsilon:  0.11
episode:  451  episode reward:  215.08  rolling reward:  127.45  Steps:  381  epsilon:  0.10
episode:  452  episode reward:  275.59  rolling reward:  128.08  Steps:  461  epsilon:  0.10
episode:  453  episode reward:  116.07  rolling reward:  127.08  Step

episode:  532  episode reward:  215.82  rolling reward:  209.89  Steps:  386  epsilon:  0.07
episode:  533  episode reward:  54.46  rolling reward:  209.84  Steps:  184  epsilon:  0.07
episode:  534  episode reward:  62.97  rolling reward:  207.76  Steps:  162  epsilon:  0.07
episode:  535  episode reward:  -4.01  rolling reward:  205.27  Steps:  1000  epsilon:  0.07
episode:  536  episode reward:  285.34  rolling reward:  205.44  Steps:  217  epsilon:  0.07
episode:  537  episode reward:  276.98  rolling reward:  208.28  Steps:  263  epsilon:  0.07
episode:  538  episode reward:  258.41  rolling reward:  210.81  Steps:  253  epsilon:  0.07
episode:  539  episode reward:  263.74  rolling reward:  213.30  Steps:  493  epsilon:  0.07
episode:  540  episode reward:  -24.97  rolling reward:  211.29  Steps:  347  epsilon:  0.07
episode:  541  episode reward:  249.35  rolling reward:  213.99  Steps:  297  epsilon:  0.07
episode:  542  episode reward:  243.97  rolling reward:  213.89  Steps: 

In [ ]:
online_model.save_weights('weights_singleDQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5')

import pickle

with open('weights_singleDQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)


In [167]:
rward = pickle.load( open( 'weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', "rb" ) )

In [168]:
rward

{'rewards': [-131.3223570990876, -248.23258991485326, -133.48636655895385],
 'rolling_reward': [-131.3223570990876,
  -189.77747350697044,
  -171.01377119096492]}

In [163]:
episode_reward

[]

In [152]:
Y_target.shape

(5, 4)

In [142]:
batch[terminal_samples].shape

(0, 5)

In [125]:



# i:index for sample [state,action,reward,new_state,done]
for i,sample in enumerate(batch):
    terminal_sample = sample[4]
    new_state_sample = sample[3]
    state_sample = sample[0]
    X[i]=state_sample
    action_sample = sample[1]
    Y_target[i] = online_model.predict(state_sample.reshape(1,-1))[0]
    if terminal:
        #terminal then targetY is reward
        Y_target[i][action_sample] = sample[2]
    else:
        #use online weights to select argmax action under new_state

        online_qvalues_batch =online_model.predict(new_state_sample.reshape(1,-1)) 
        max_action = np.argmax(online_qvalues_batch)
        #use target weights to calculate Qvalue
        #non-terminal then targetY is reward +gamma*new Qvalue
        Y_target[i][action_sample] = sample[2] + gamma * target_model.predict(new_state_sample.reshape(1,-1)).reshape(4,)[max_action]

online_model.fit(np.array(X), np.array(Y_target), verbose=0)


100

In [ ]:
Notice that the selection of the action, in the argmax, is


In [91]:
online_qvalues_batch

[array([[ 0.18740644, -0.2721934 ,  0.2233217 , -0.45678362]],
       dtype=float32),
 array([[ 0.19613695, -0.27545747,  0.20470577, -0.46681872]],
       dtype=float32),
 array([[ 0.18746991, -0.25133634,  0.19952038, -0.4359238 ]],
       dtype=float32),
 array([[ 0.19334945, -0.2700162 ,  0.19816679, -0.45707738]],
       dtype=float32),
 array([[ 0.19308917, -0.27386588,  0.21208042, -0.46272066]],
       dtype=float32)]

In [113]:
np.array(X)

array([[-0.00213823,  1.4003743 , -0.21659775, -0.4687014 ,  0.0024845 ,
         0.04906261,  0.        ,  0.        ],
       [-0.00846777,  1.3651963 , -0.20548455, -0.5481141 ,  0.00758275,
         0.00502167,  0.        ,  0.        ],
       [-0.01887293,  1.3004851 , -0.22156799, -0.61512053,  0.00625335,
         0.04479944,  0.        ,  0.        ],
       [-0.01250668,  1.3402183 , -0.20310256, -0.5355056 ,  0.00488083,
        -0.02924636,  0.        ,  0.        ],
       [-0.00641537,  1.3775289 , -0.21631236, -0.5210343 ,  0.00733169,
         0.04846335,  0.        ,  0.        ]], dtype=float32)

In [112]:
Y_target

[array([-1.2552428 , -0.2741652 ,  0.2263453 , -0.45686692], dtype=float32),
 array([ 0.19308917, -0.27386588,  0.21208042, -0.6722371 ], dtype=float32),
 array([ 0.1933487, -0.2546284,  5.1014786, -0.445849 ], dtype=float32),
 array([-0.91460276, -0.27277392,  0.20281965, -0.4571485 ], dtype=float32),
 array([ 0.18904   , -0.27001685,  0.21992686, -0.7534547 ], dtype=float32)]

In [ ]:

class DDQL:
    def __init__(self, nS, nA):
        self.nS = nS
        self.nA = nA
        self.epsilon = 1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9993
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.epochs = 1
        self.verbose = 0
        self.minibatch_size = 30
        self.memory = deque(maxlen=5000)
        self.model = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()

        # Add 2 hidden layers with 64 nodes each
        model.add(Dense(64, input_dim=self.nS, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.nA, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, s, a, r, s_prime, done):
        self.memory.append((s, a, r, s_prime, done))

    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())

    def selectAction(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.nA)
        q = self.model.predict(s)
        return np.argmax(q[0])

    def replay(self):
        # Vectorized method for experience replay
        minibatch = random.sample(self.memory, self.minibatch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])

        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(self.minibatch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=self.epochs, verbose=self.verbose)

    def replayIterative(self):
        # Iterative method - this performs the same function as replay() but is not vectorized 
        s_list = []
        y_state_list = []
        minibatch = random.sample(self.memory, self.minibatch_size)
        for s, a, r, s_prime, done in minibatch:
            s_list.append(s)
            y_action = r
            if not done:
                y_action = r + self.gamma * np.amax(self.model.predict(s_prime)[0])

            print(y_action)
            
            y_state = self.model.predict(s)
            y_state[0][a] = y_action
            y_state_list.append(y_state)
        self.model.fit(np.squeeze(s_list), np.squeeze(y_state_list), batch_size=self.minibatch_size, epochs=1, verbose=0)

if __name__ == "__main__":
    np.set_printoptions(precision=2)
    env = gym.make('LunarLander-v2')
    env = wrappers.Monitor(env, tdir, force=True, video_callable=False)

    nS = env.observation_space.shape[0]
    nA = env.action_space.n

    agent = DDQL(nS, nA)

    # Set to true to use saved model
    episodes = 10000

    # Cumulative reward
    reward_avg = deque(maxlen=100)

    for e in range(episodes):
        episode_reward = 0
        s = env.reset()
        s = np.reshape(s, [1, nS])

        for time in range(1000):

            # Query next action from learner and perform action
            a = agent.selectAction(s)
            s_prime, r, done, info = env.step(a)

            # Add cumulative reward
            episode_reward += r

            # Reshape new state
            s_prime = np.reshape(s_prime, [1, nS])

            # Add experience to memory
            agent.add_memory(s, a, r, s_prime, done)

            # Set current state to new state
            s = s_prime

            #Perform experience replay if memory length is greater than minibatch length
   
            if len(agent.memory) > agent.minibatch_size:
                agent.replay()

            # If episode is done, exit loop
            if done:
                agent.target_model_update()
                break


        # epsilon decay
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay

        # Running average of past 100 episodes
        reward_avg.append(episode_reward)
        print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)
        
        # with open('trained_agent.txt', 'a') as f:
        #     f.write(str(np.average(reward_avg)) + '\n')

    env.close()
#     gym.upload(tdir, api_key='sk_EJo79Jo0RsSEA4EF6Cp5mg')


#agent.model.save_weights('weights.h5')
#agent.model.load_weights('weights.h5')

episode:  0  score:  -101.20  avg_score:  -101.20  frames:  104  epsilon:  1.00
episode:  1  score:  -147.99  avg_score:  -124.59  frames:  74  epsilon:  1.00
episode:  2  score:  -95.67  avg_score:  -114.95  frames:  74  epsilon:  1.00
episode:  3  score:  -218.30  avg_score:  -140.79  frames:  94  epsilon:  1.00
episode:  4  score:  -231.98  avg_score:  -159.03  frames:  86  epsilon:  1.00
episode:  5  score:  -161.32  avg_score:  -159.41  frames:  68  epsilon:  1.00
episode:  6  score:  -96.64  avg_score:  -150.44  frames:  66  epsilon:  1.00
episode:  7  score:  -141.43  avg_score:  -149.32  frames:  97  epsilon:  0.99
episode:  8  score:  -480.81  avg_score:  -186.15  frames:  102  epsilon:  0.99
episode:  9  score:  -371.18  avg_score:  -204.65  frames:  92  epsilon:  0.99
episode:  10  score:  -161.56  avg_score:  -200.73  frames:  95  epsilon:  0.99
episode:  11  score:  -80.20  avg_score:  -190.69  frames:  61  epsilon:  0.99
episode:  12  score:  -332.47  avg_score:  -201.60 

episode:  103  score:  -168.96  avg_score:  -179.65  frames:  118  epsilon:  0.93
episode:  104  score:  -64.43  avg_score:  -177.97  frames:  91  epsilon:  0.93
episode:  105  score:  -200.70  avg_score:  -178.36  frames:  72  epsilon:  0.93
episode:  106  score:  32.45  avg_score:  -177.07  frames:  121  epsilon:  0.93
episode:  107  score:  -260.25  avg_score:  -178.26  frames:  117  epsilon:  0.93
episode:  108  score:  -110.97  avg_score:  -174.56  frames:  80  epsilon:  0.93
episode:  109  score:  -253.15  avg_score:  -173.38  frames:  94  epsilon:  0.93
episode:  110  score:  -125.12  avg_score:  -173.02  frames:  88  epsilon:  0.93
episode:  111  score:  -189.74  avg_score:  -174.11  frames:  72  epsilon:  0.92
episode:  112  score:  -140.07  avg_score:  -172.19  frames:  94  epsilon:  0.92
episode:  113  score:  -304.20  avg_score:  -173.17  frames:  71  epsilon:  0.92
episode:  114  score:  -128.94  avg_score:  -173.02  frames:  72  epsilon:  0.92
episode:  115  score:  -294.

episode:  204  score:  -298.51  avg_score:  -150.87  frames:  97  epsilon:  0.87
episode:  205  score:  -77.35  avg_score:  -149.64  frames:  146  epsilon:  0.87
episode:  206  score:  -124.22  avg_score:  -151.21  frames:  68  epsilon:  0.87
episode:  207  score:  -40.81  avg_score:  -149.01  frames:  144  epsilon:  0.86
episode:  208  score:  -83.08  avg_score:  -148.73  frames:  67  epsilon:  0.86
episode:  209  score:  -268.33  avg_score:  -148.89  frames:  92  epsilon:  0.86
episode:  210  score:  -124.07  avg_score:  -148.88  frames:  105  epsilon:  0.86
episode:  211  score:  -123.48  avg_score:  -148.21  frames:  122  epsilon:  0.86
episode:  212  score:  -143.46  avg_score:  -148.25  frames:  80  epsilon:  0.86
episode:  213  score:  -86.07  avg_score:  -146.07  frames:  96  epsilon:  0.86
episode:  214  score:  -86.35  avg_score:  -145.64  frames:  94  epsilon:  0.86
episode:  215  score:  -55.86  avg_score:  -143.25  frames:  93  epsilon:  0.86
episode:  216  score:  -58.61 